In [2]:
#get records and check matches
import pymongo
import json
from collections import Iterable
import pandas as pd

connection = pymongo.MongoClient('localhost', 27017)

db = connection['my_phd']

collection = db['clustered_collections']

#function to combine the nested list with the parent one (https://stackoverflow.com/questions/952914/how-do-i-make-a-flat-list-out-of-a-list-of-lists)
def flatten(items):
    """Yield items from any nested iterable; see Reference."""
    for x in items:
        if isinstance(x, Iterable) and not isinstance(x, (str, bytes)):
            for sub_x in flatten(x):
                yield sub_x
        else:
            yield x

#get the composed service
def get_composed_service_attributes():
    batch = collection.find({"Anchor": "Composed service"})
    batch_output = []
    for item in batch:
        output = []
        #print(item)
        for element in item["Features"]:
            for each in element["Features"]:
                output.append([each["type"], each["attribute"]])
    
        batch_output.append(output)    
    return(list(batch_output))            
        
#get the devices 
def get_device_attributes():
    batch = collection.find({"Anchor": "Device"})
    
    output = []
    for item in batch:
        output.append([item["type"], item["attribute"]])
    return(output)

def attributes_matching():
    composed_service_attributes = get_composed_service_attributes()
    devices_attributes = get_device_attributes()

    #iterrate over the available services
    result = []
    for service in composed_service_attributes:
        #iterrate over attributes of available services
        for attr_ser in service:
            #iterrate over attributes of available devices
            for attr_dev in devices_attributes:
                #check if the filed type of device is the same as needed by the service
                type_match = attr_ser[::2] == attr_dev[::2]
                #if the answer is True then proceed
                if type_match == True:
                    attr_match = set(list(flatten(attr_ser[1::2]))).isdisjoint(set(list(flatten(attr_dev[1::2]))))
                    #check if there any attrbiute in service is in the device description
                    if attr_match == False:
                        #get the similarity score, i.e. how close the device satisfy the service attribute conditions
                        suitability_score = (1 - (len(list(set(flatten(attr_ser[1::2])) - set(flatten(attr_dev[1::2])))) / len(list(flatten(attr_ser[1::2])))))*100
                        result.append(['device with the following attributes: ', attr_dev, ' satisfies the service demand ', attr_ser, ' to the following extent: ', suitability_score, '%'])
                        
    return([list(flatten(item)) for item in result])
        

attributes_matching()

C:\Users\Artem\AppData\Local\Temp\ipykernel_20988\2393547835.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Iterable


[['device with the following attributes: ',
  'sensor',
  'temperature',
  'humidity',
  ' satisfies the service demand ',
  'sensor',
  'temperature',
  ' to the following extent: ',
  100.0,
  '%'],
 ['device with the following attributes: ',
  'sensor',
  'temperature',
  'humidity',
  ' satisfies the service demand ',
  'sensor',
  'humidity',
  ' to the following extent: ',
  100.0,
  '%'],
 ['device with the following attributes: ',
  'sensor',
  'temperature',
  'humidity',
  ' satisfies the service demand ',
  'sensor',
  'temperature',
  'pressure',
  ' to the following extent: ',
  50.0,
  '%'],
 ['device with the following attributes: ',
  'sensor',
  'temperature',
  'humidity',
  ' satisfies the service demand ',
  'sensor',
  'humidity',
  ' to the following extent: ',
  100.0,
  '%']]